<a href="https://colab.research.google.com/github/heidingaway/datapeople/blob/main/GCThesaurus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [113]:
import pandas as pd
import os
import zipfile
import yaml
import re
import unicodedata

In [114]:
## Link to CSV

src = "https://canada.multites.net/cst/EAEAD1E6-7DD2-4997-BE7F-40BFB1CBE8A2/CST20240911.csv"

# Create the DataFrame from all the retrieved records
df = pd.read_csv(src)

# Add column names
df.columns = ['subject', 'predicate', 'object']

In [115]:
# Create the DataFrame from all the retrieved records
print(df.head())

                          subject predicate                object
0               2-spirited people       Use     Two-spirit people
1               2019-nCoV disease       Use  Coronavirus diseases
2  2019 novel coronavirus disease       Use  Coronavirus diseases
3                 2SLGBTQ+ people       Use      2SLGBTQI+ people
4                2SLGBTQI+ people    French    Personne 2ELGBTQI+


In [116]:
# Define the strings you want to add
prefix = "[["
suffix = "]]"

# Use the apply method with a lambda function to modify the column content
df['linkedsubject'] = df['subject'].apply(lambda x: f"{prefix}{x}{suffix}")

# Define the headings you want to add
heading2 = "## "

# Use the apply method with a lambda function to modify the column content
df['h2'] = df['predicate'].apply(lambda x: f"{heading2}{x}")

# identify title
df = df.rename(columns={'object': 'title'})

# Display the modified DataFrame
print(df.head())

                          subject predicate                 title  \
0               2-spirited people       Use     Two-spirit people   
1               2019-nCoV disease       Use  Coronavirus diseases   
2  2019 novel coronavirus disease       Use  Coronavirus diseases   
3                 2SLGBTQ+ people       Use      2SLGBTQI+ people   
4                2SLGBTQI+ people    French    Personne 2ELGBTQI+   

                        linkedsubject         h2  
0               [[2-spirited people]]     ## Use  
1               [[2019-nCoV disease]]     ## Use  
2  [[2019 novel coronavirus disease]]     ## Use  
3                 [[2SLGBTQ+ people]]     ## Use  
4                [[2SLGBTQI+ people]]  ## French  


In [117]:
# Group by 'h2' and then 'title' in a single groupby operation
grouped = df.groupby(['h2', 'title'])

# Print the sorted results
print(grouped.head().sort_values(by='title', ascending=True))

                    subject     predicate  \
4789         Cycling trails  History note   
2016          Bicycle paths  History note   
2082   Biochemical products  History note   
2090           Biochemicals  History note   
3436                 Coasts  History note   
...                     ...           ...   
19712         Tax avoidance        French   
4635        Cultural events        French   
19251         Sports events        French   
10161          Human beings        French   
1475              Art works        French   

                                                   title  \
4789   "Bicycle paths" replaces "Cycling trails" as p...   
2016   "Bicycle paths" replaces "Cycling trails" as p...   
2082   "Biochemicals" replaces "Biochemical products"...   
2090   "Biochemicals" replaces "Biochemical products"...   
3436   "Coasts" replaces "Coastlands" as preferred te...   
...                                                  ...   
19712                                  

In [118]:
# Create a directory to store the CSV files
output_dir = 'csv_files8'  # Name of the directory
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

# Save as csv, using the title column for file names (sanitized)
def save_to_csv(x):
    # Get the 'title' from the first row of the group
    title = x['title'].iloc[0]

    # Sanitize the title to create a safe file name
    file_name = "".join([c for c in title if c.isalnum() or c in "._- "]).rstrip()[:100] + ".csv"  # Limit to 100 characters

    # Create the full file path
    file_path = os.path.join(output_dir, file_name)

    # Check if file already exists, add a counter if needed
    counter = 1
    while os.path.exists(file_path):
        file_name = "".join([c for c in title if c.isalnum() or c in "._- "]).rstrip()[:100] + f"_{counter}.csv"
        file_path = os.path.join(output_dir, file_name)
        counter += 1

    x.to_csv(file_path, index=False)

grouped.apply(save_to_csv)
print(f"DataFrame saved to multiple files in '{output_dir}' directory")

DataFrame saved to multiple files in 'csv_files8' directory


<ipython-input-118-07f27c7900f6>:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped.apply(save_to_csv)
